In [1]:
# import pandas as pd
# import numpy as np
# import warnings


# pd.options.mode.chained_assignment = None
# pd.set_option('display.max_columns', 80)
# pd.set_option('display.max_rows', 55)

# warnings.simplefilter(action='ignore', category=FutureWarning)

# item_master_excel_file = '../data/inzi/Item Master_08_18_2021 0317.xls'
# material_history_excel_file = '../data/inzi/Material Movement History_08_19_2021 1026.xls'
# out_put_file = '../output/9-15.xlsx'

# item_master = pd.read_excel(
#     item_master_excel_file, index_col=False, header=1
# )
# material_history = pd.read_excel(
#     material_history_excel_file, index_col=False, header=1
# )

# # Item master excel file must have 3 columns 'Material', 'Material Type', 'Procurement'
# temp_item_master = item_master[['Material', 'Material Type', 'Procurement']]

# '''
# PREPARE FOR STEP ONE
# '''
# step_one = pd.merge(material_history, temp_item_master,
#                     on='Material', how='left')

# step_one.rename(columns={
#     'Reference': 'Order Category',
#     'Unnamed: 13': 'Order Data',
#     'Unnamed: 14': 'Master Category',
#     'Unnamed: 15': 'Master Data',
#     'Unnamed: 16': 'Remark'
# }, inplace=True)

# step_one = step_one.iloc[1:, :]

# # pd.set_option('mode.chained_assignment', None)

# step_one.loc[step_one['Quantity'] > 0, 'Input'] = step_one['Quantity']
# step_one.loc[step_one['Quantity'] > 0, 'Output'] = 0
# step_one.loc[step_one['Quantity'] <= 0, 'Input'] = 0
# step_one.loc[step_one['Quantity'] <= 0, 'Output'] = -step_one['Quantity']

# cols = step_one.columns.tolist()

# cols = cols[:10] + [cols[-2], cols[-1]] + cols[10:-2]

# step_one = step_one[cols]

# step_one['Account code'] = [int(x)
#                             for x in (step_one['Movement Type'].str[:3])]

# cols = step_one.columns.tolist()
# cols = cols[:7] + [cols[-1]] + cols[7: -1]
# step_one = step_one[cols]


In [2]:
# steady = step_one.pivot_table(index=['Unnamed: 0', 'Material'],
#                             columns='Account code', values=['Quantity'], aggfunc=np.sum, fill_value=0)

# steady.reset_index(inplace=True)
# del steady['Unnamed: 0']
# steady = steady.groupby(steady['Material']).aggregate(
#         'sum')

# steady['IN'] = steady.where(steady > 0).sum(1)
# steady['OUT'] = steady.where(steady < 0).sum(1)

# steady_2 = steady.iloc[:, [-2, -1]]


In [3]:
# from functools import reduce

# def merge_result(dfs):
#     df_final = reduce(lambda left,right: pd.merge(left, right, how="outer", on=['Material'], sort=False,
#                     indicator=False, validate=None, suffixes=('_01', '_02',)), 
#                     dfs)
#     return df_final

# stock = pd.read_excel('../data/inzi/stock.xlsb', header=2)
# stock.rename(columns={
#     'Unnamed: 5': 'Materials'
# }, inplace=True)
# stock['OPENING STOCK'] = stock['Unrestricted'] + stock['Inspection'] + stock['Returned'] + stock['Blocked']
# stock = stock[['Materials', 'OPENING STOCK']]
# stock.rename(columns={'Materials': 'Material'}, inplace=True)

# temp = pd.merge(stock, aug_28, how="outer", on=['Material'], sort=False,
#                     indicator=False, validate=None)



# temp_aug = pd.merge(aug_28, stock, how="outer", on=['Material'], sort=False,
#                     indicator=False, validate=None)

# temp_aug

# temp_aug.set_axis([*temp_aug.columns[:-1], 'temporery'], axis=1, inplace=True)

# temp_aug['CLOSING_STOCK'] = temp_aug[[101, 102, 'temporery']].sum(axis=1) - temp_aug[[401, 720]].sum(axis=1)

# temp_aug[temp_aug.columns[[0, -1]]]

# dfs = [aug_28, aug_29, aug_28, aug_29, aug_28, aug_29, aug_28, aug_29, aug_28]
# result = pd.concat(dfs)
# result['temp'] = 0

# result = result[['Material', 'temp']]

# result.drop_duplicates(inplace=True)

# final_result = pd.concat(dfs)
# final_result['temp'] = 0
# final_result = final_result[['Material', 'temp']]
# final_result.drop_duplicates(inplace=True)

# final_result = pd.merge(final_result, stock[stock.columns[[0, -1]]], how="left", on=['Material'], sort=False,
#                         indicator=False, validate=None)

In [4]:
# import numpy as np
# from functools import reduce
import warnings

import pandas as pd

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 55)

warnings.simplefilter(action='ignore', category=FutureWarning)

open_stock = '../data/inzi/stock.xlsb'

files = [
    '../data/inzi/02 SEP/01-09-2021_IN-OUT DAILY REPORT.xlsb',
    '../data/inzi/03 SEP/02-09-2021_IN-OUT DAILY REPORT.xlsb',
    '../data/inzi/04 SEP/03-09-2021_IN-OUT DAILY REPORT.xlsb',
]

out_put_file = '../output/concet.xlsx'


stock = pd.read_excel(open_stock, header=2)

stock.rename(columns={
    'Unnamed: 5': 'Materials'
}, inplace=True)
stock['OPENING STOCK'] = stock['Unrestricted'] + \
    stock['Inspection'] + stock['Returned'] + stock['Blocked']
stock = stock[['Materials', 'OPENING STOCK']]
stock.rename(columns={'Materials': 'Material'}, inplace=True)
stock = stock.groupby(stock['Material']).aggregate(
    'sum')


def get_concat(sheet):
    dfs = []
    for file_name in files:
        df = pd.read_excel(file_name, sheet_name=sheet, header=8)
        df.drop(columns=['Unnamed: 0', 'Unnamed: 2',
                         'IN', 'OUT'], inplace=True)
        df.rename(columns={'Unnamed: 1': 'Material'}, inplace=True)
        if 'Unnamed: 5' in df:
            df.rename(columns={'Unnamed: 5': 'PURCHASE'}, inplace=True)
        dfs.append(df)

    final_result = pd.concat(dfs)
    final_result['temp'] = 0
    final_result = final_result[['Material', 'temp']]
    final_result.drop_duplicates(inplace=True)
    final_result = pd.merge(final_result, stock, how="left", on=['Material'], sort=False,
                            indicator=False, validate=None)
    del final_result['temp']
    l = len(dfs)

    for _ in range(l):
        in_stock = [344, 401, 623, 720, 321, 323, 327, 602]
        sum_col = []
        out_stock = [401, 720]
        subtract_col = []

        dfs[0] = pd.merge(dfs[0], final_result[final_result.columns[[0, -1]]], how="outer", on=['Material'], sort=False,
                          indicator=False, validate=None)
        dfs[0].set_axis([*dfs[0].columns[:-1], 'temporary'],
                        axis=1, inplace=True)

        for column in in_stock:
            if column in dfs[0].columns:
                sum_col.append(column)
        sum_col.append('temporary')

        for column in out_stock:
            if column in dfs[0].columns:
                subtract_col.append(column)

        dfs[0]['CLOSING STOCK'] = dfs[0][sum_col].sum(
            axis=1) - dfs[0][subtract_col].sum(axis=1)
        del dfs[0]['temporary']

        final_result = pd.merge(final_result, dfs[0], how="outer", on=['Material'], sort=False,
                                indicator=False, validate=None)
        del dfs[0]
    print(f'Concat {l} files together in {sheet}')
    # final_result.drop(final_result.columns[-1], axis=1, inplace=True)
    final_result.fillna(0, inplace=True)
    return final_result


raw_concat = get_concat('REPORT-RAW')
wip_concat = get_concat('REPORT-WIP')
fg_concat = get_concat('REPORT-FG')

Concat 3 files together in REPORT-RAW
Concat 3 files together in REPORT-WIP
Concat 3 files together in REPORT-FG


In [5]:
check_case_one = pd.merge(raw_concat, fg_concat, how="inner", on=['Material'], sort=False,
                                indicator=False, validate=None)

In [6]:
check_case_one

,Material,OPENING STOCK_x,101_x_x,102_x,PURCHASE_x,321_x_x,344_x_x,401_x_x,623_x_x,720_x_x,201_x_x,261_x_x,321.1_x_x,344.1_x_x,401.1_x_x,601_x_x,609_x_x,721_x_x,CLOSING STOCK_x_x,101_y_x,102_y,PURCHASE_y,321_y_x,323_x,327_x_x,344_y_x,401_y_x,623_y_x,720_y_x,201_y_x,261_y_x,321.1_y_x,323.1_x,327.1_x_x,344.1_y_x,401.1_y_x,601_y_x,609_y_x,721_y_x,CLOSING STOCK_y_x,...,721_x_y,Unnamed: 19,CLOSING STOCK_x_y,101_y_y,321_y_y,323_y,327_x_y,344_y_y,401_y_y,623_y_y,720_y_y,201_y_y,261_y_y,321.1_y_y,323.1_y,327.1_x_y,344.1_y_y,401.1_y_y,601_y_y,609_y_y,721_y_y,CLOSING STOCK_y_y,101_y,262_y,321_y,327_y_y,344_y,401_y,623_y,720_y,201_y,261_y,321.1_y,327.1_y_y,344.1_y,401.1_y,601_y,609_y,721_y,CLOSING STOCK_y
0,BN96-45687A,7513.0,5760.0,0.0,5760.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7513.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7513.0,...,0.0,0.0,7513.0,524.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.0,0.0,0.0,7513.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,524.0,0.0,0.0,7513.0
1,BN96-48389A,1941.0,1236.0,0.0,1236.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1941.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1941.0,...,0.0,0.0,1941.0,720.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1236.0,1941.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,720.0,1941.0
2,BN96-45687B,478.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,478.0,524.0,0.0,524.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,478.0,...,0.0,0.0,478.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,478.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,301.0,0.0,0.0,478.0
3,BN96-45688A,3221.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3221.0,202.0,0.0,202.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3221.0,...,0.0,0.0,3221.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3221.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,162.0,0.0,0.0,3221.0
4,BN96-45689A,770.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,770.0,21.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,770.0,...,0.0,0.0,770.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,770.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,770.0
5,BN96-45690A,5460.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5460.0,162.0,0.0,162.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5460.0,...,0.0,0.0,5460.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5460.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,162.0,0.0,0.0,5460.0
6,BN96-45691A,351.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,351.0,21.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,351.0,...,0.0,0.0,351.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,351.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,0.0,0.0,351.0
7,BN96-53186C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,130.0,0.0,130.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,130.0,0.0,0.0,0.0
8,HS02-00156A,457.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,457.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,457.0,...,0.0,0.0,457.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,457.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,457.0


In [7]:
goods_processing = pd.read_excel('../data/inzi/goods processing.xlsx', sheet_name=None)
for index, row in check_case_one.iterrows():
    material = row['Material']
    for name, sheet in goods_processing.items():
        sheet = sheet[['Unnamed: 1', 'Unnamed: 12']]
        sheet.rename(columns={
            'Unnamed: 1': 'Trading', 'Unnamed: 12': 'Processing',
        }, inplace=True)
        if material in sheet['Trading'].unique():
            print(f'Item {material} in trading of {name}')
        elif material in sheet['Processing'].unique():
            print(f'Item {material} in processing of {name}')
        else:
            pass
    

Item BN96-45687A in processing of GNI VN
Item BN96-48389A in trading of TOHO
Item BN96-45687B in processing of GNI VN
Item BN96-45688A in processing of GNI VN
Item BN96-45689A in processing of GNI VN
Item BN96-45690A in processing of GNI VN
Item BN96-45691A in processing of GNI VN
Item BN96-53186C in processing of GNI VN


In [8]:
fg_concat = fg_concat.drop(fg_concat[fg_concat['Material'] == 'BN96-45687A'].index)

In [9]:
raw_concat = raw_concat.append(fg_concat[fg_concat['Material'] == 'BN96-45688A'])

In [10]:
raw_concat.columns = [str(x)[0:3] if str(x)[0:3].isdigit() else str(x).split('_')[0] for x in raw_concat.columns]

In [ ]:
raw_concat = raw_concat.append(fg_concat[fg_concat['Material'] == 'BN96-45688A'])

In [11]:
raw_concat

,Material,OPENING STOCK,101,102,PURCHASE,321,344,401,623,720,201,261,321,344,401,601,609,721,CLOSING STOCK,101,102,PURCHASE,321,323,327,344,401,623,720,201,261,321,323,327,344,401,601,609,721,CLOSING STOCK,101,102,PURCHASE,262,321,327,344,401,623,720,201,261,321,327,344,401,601,609,721,CLOSING STOCK,Unnamed: 19
0,0103-004609,4405179.63,2550000.0,0.0,2550000.0,0.0,0.0,1200000.0,0.0,0.0,0.0,217442.8,0.0,0.0,1200000.0,0.0,0.0,0.0,4405179.63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,850000.0,0.0,0.0,0.0,435425.0,0.0,0.0,0.0,0.0,850000.0,0.0,0.0,0.0,4405179.63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400000.0,0.0,0.0,0.0,383100.0,0.0,0.0,0.0,400000.0,0.0,0.0,0.0,4405179.63,NaN
1,0103-007368,5256854.00,2400000.0,0.0,2400000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5256854.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40000.0,0.0,0.0,0.0,33600.0,0.0,0.0,0.0,0.0,40000.0,0.0,0.0,0.0,5256854.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5256854.00,NaN
2,0103-007683,308340.00,0.0,0.0,0.0,0.0,0.0,150000.0,0.0,0.0,0.0,113820.0,0.0,0.0,150000.0,0.0,0.0,0.0,308340.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,150000.0,0.0,0.0,0.0,196380.0,0.0,0.0,0.0,0.0,150000.0,0.0,0.0,0.0,308340.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18450.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,308340.00,NaN
3,0103-009882,7961700.00,850000.0,0.0,850000.0,0.0,0.0,100000.0,0.0,0.0,0.0,100000.0,0.0,0.0,100000.0,0.0,0.0,0.0,7961700.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125000.0,0.0,0.0,0.0,100900.0,0.0,0.0,0.0,0.0,125000.0,0.0,0.0,0.0,7961700.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,425000.0,0.0,0.0,0.0,5600.0,0.0,0.0,0.0,425000.0,0.0,0.0,0.0,7961700.00,NaN
4,0103-010237,1394369.00,0.0,0.0,0.0,0.0,0.0,990000.0,0.0,0.0,0.0,842640.0,0.0,0.0,990000.0,0.0,0.0,0.0,1394369.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,700000.0,0.0,0.0,0.0,490770.0,0.0,0.0,0.0,0.0,700000.0,0.0,0.0,0.0,1394369.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200000.0,0.0,0.0,0.0,178980.0,0.0,0.0,0.0,200000.0,0.0,0.0,0.0,1394369.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,BN96-54189A,1194.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1194.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1194.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95.0,0.0,0.0,0.0,95.0,0.0,0.0,0.0,95.0,0.0,0.0,0.0,1194.00,NaN
475,BN96-54190A,1194.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1194.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1194.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95.0,0.0,0.0,0.0,95.0,0.0,0.0,0.0,95.0,0.0,0.0,0.0,1194.00,NaN
476,HS02-00156A,457.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,457.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,457.00,1300.0,0.0,1300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,457.00,NaN
477,IZ01-000035,3308.94,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3308.94,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3308.94,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1056.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3308.94,NaN
